# Глубинное обучение

Классические модели машинного обучения показали ограниченную эффективность при решении задачи классификации направления ценового движения на основе финансовых текстов. Одной из причин являются ограничения в обработке контекста и последовательностей слов, которые играют ключевую роль в анализе текстовой информации.

Глубинное обучение позволяет учитывать сложные языковые зависимости и взаимодействия признаков. В этой части исследования реализуем три модели нейронных сетей с использованием PyTorch.
Для ускорения обучения и сохранения интерпретируемости выбраны лёгкие архитектуры: TextCNN, biGRU и FastText-style. Для всех моделей используем те же признаки и целевую переменную, что и ранее, но на увеличенном датасете (50 000 строк). Метрики оценки - accuracy и F1 score.


In [7]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available!
Device Name: NVIDIA GeForce RTX 3080 Ti


In [13]:
# Подготовка датасета из 50 тыс. строк с равномерным распределением целевой переменной

import pandas as pd
import glob
from sklearn.utils import resample

RANDOM_SEED = 42

files = sorted(glob.glob('000[0-6].parquet'))
df = pd.concat([pd.read_parquet(f, engine='pyarrow') for f in files], ignore_index=True)

df = df.iloc[:200_000]

df['price_24h_change_percent'] = ((df['weighted_avg_24_hrs'] - df['weighted_avg_0_hrs']) / df['weighted_avg_0_hrs'] * 100).round(2)

df['target'] = (df['price_24h_change_percent'] > 0).astype(int)

df_0 = df[df['target'] == 0]
df_1 = df[df['target'] == 1]

df_0_sample = resample(df_0, replace=False, n_samples=25_000, random_state=RANDOM_SEED)
df_1_sample = resample(df_1, replace=False, n_samples=25_000, random_state=RANDOM_SEED)

df = pd.concat([df_0_sample, df_1_sample]).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

print(df['target'].value_counts())

target
1    25000
0    25000
Name: count, dtype: int64


In [17]:
# Предобработка - убираем пустые значения, добавляем поле text
df = df.dropna(subset=['Article', 'weighted_avg_12_hrs'])
df['text'] = (df['Title'] + ' ' + df['Article']).fillna('')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Symbol                    50000 non-null  object 
 1   Security                  50000 non-null  object 
 2   Sector                    49059 non-null  object 
 3   Industry                  49059 non-null  object 
 4   URL                       50000 non-null  object 
 5   Date                      50000 non-null  object 
 6   RelatedStocksList         36910 non-null  object 
 7   Article                   50000 non-null  object 
 8   Title                     49915 non-null  object 
 9   articleType               50000 non-null  object 
 10  Publication               49958 non-null  object 
 11  Author                    34490 non-null  object 
 12  weighted_avg_-96_hrs      50000 non-null  float64
 13  weighted_avg_-48_hrs      50000 non-null  float64
 14  weight

,Symbol,Security,Sector,Industry,URL,Date,RelatedStocksList,Article,Title,articleType,...,weighted_avg_12_hrs,weighted_avg_24_hrs,weighted_avg_48_hrs,weighted_avg_72_hrs,weighted_avg_96_hrs,weighted_avg_360_hrs,weighted_avg_720_hrs,price_24h_change_percent,target,text
0,DAN,Dana Incorporated,Consumer Discretionary,Auto Parts:O.E.M.,https://www.nasdaq.com/articles/dan-crosses-be...,"Mar 14, 2019 11:48 AM ET",Markets,"In trading on Thursday, shares of Dana Inc (Sy...",DAN Crosses Below Key Moving Average Level,News,...,18.0960,18.1922,18.2424,18.2528,18.4231,17.6645,20.5289,0.66,1,DAN Crosses Below Key Moving Average Level In ...
1,FCF,First Commonwealth Financial Corporation,Finance,Major Banks,https://www.nasdaq.com/articles/validea-peter-...,"Feb 22, 2024 05:34 AM ET",Markets|ROCK|VC,The following are today's upgrades for Validea...,Validea Peter Lynch Strategy Daily Upgrade Rep...,News,...,13.1567,13.1727,13.2255,13.1243,13.1187,13.6013,13.5000,-0.27,0,Validea Peter Lynch Strategy Daily Upgrade Rep...
2,CGC,Canopy Growth Corporation,Health Care,Medicinal Chemicals and Botanical Products,https://www.nasdaq.com/articles/cronos-group-3...,"Apr 08, 2019 05:23 PM ET",CRON|Markets|TLRY|ACB|MO,"**Cronos Group** (NASDAQ:), a top Canadian can...","Cronos Group: 3 Pros, 3 Cons for Buying CRON S...",News,...,42.6443,42.4592,41.9311,42.0994,41.4073,48.2890,47.4887,-2.79,0,"Cronos Group: 3 Pros, 3 Cons for Buying CRON S..."
3,ECPG,"Encore Capital Group, Inc.",Finance,Finance Companies,https://www.nasdaq.com/articles/encore-capital...,"Jan 15, 2022 12:33 AM ET",Stocks,"Encore Capital Group, Inc. ([ECPG](https://kwh...","Encore Capital Group, Inc. Shares Climb 3.9% P...",News,...,68.0376,67.7071,67.0765,66.8197,67.1355,65.3444,70.3295,-0.49,0,"Encore Capital Group, Inc. Shares Climb 3.9% P..."
4,ARCT,Arcturus Therapeutics Holdings Inc.,Health Care,Biotechnology: Pharmaceutical Preparations,https://www.nasdaq.com/press-release/arcturus-...,"Apr 13, 2020 08:01 AM ET",None,Clinical Plan Includes Healthy Volunteers in N...,Arcturus Therapeutics Announces Allowance of I...,Press Release,...,16.5429,16.8677,18.3530,17.1136,17.1753,32.4890,46.7779,9.15,1,Arcturus Therapeutics Announces Allowance of I...


In [23]:
# Делаем сентимент-анализ с VADER, добавляем в датасет
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
tqdm.pandas()

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if isinstance(text, str):
        return analyzer.polarity_scores(text)['compound']
    else:
        return 0.0

# Параллельная обработка
sentiments = Parallel(n_jobs=-1)(
    delayed(get_sentiment)(text) for text in tqdm(df['text'], desc="Sentiment Analysis")
)
df['sentiment'] = sentiments

Sentiment Analysis:   0%|          | 0/50000 [00:00<?, ?it/s]

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from collections import Counter
import numpy as np
import re

In [43]:
# Предобработка

from collections import Counter

def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.split()

# Считаем частоты токенов без создания большого списка
token_counter = Counter()
for text in df['text']:
    token_counter.update(tokenize(text))

# Строим словарь на основе самых частотных токенов
vocab = {'<PAD>': 0, '<UNK>': 1}
vocab.update({word: i+2 for i, (word, _) in enumerate(token_counter.most_common(20_000))})

def encode(text, max_len=200):
    tokens = tokenize(text)
    ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    padded = ids[:max_len] + [vocab['<PAD>']] * (max_len - len(ids))
    return padded

In [47]:
class NewsDataset(Dataset):
    def __init__(self, texts, sentiments, labels):
        self.texts = texts
        self.sentiments = sentiments
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'text': torch.tensor(self.texts[idx], dtype=torch.long),
            'sentiment': torch.tensor([self.sentiments[idx]], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Кодирование текста
encoded_texts = [encode(text) for text in df['text']]
sentiments = df['sentiment'].tolist()
labels = df['target'].tolist()

# Разделение на train/test
X_train, X_val, s_train, s_val, y_train, y_val = train_test_split(
    encoded_texts, sentiments, labels, test_size=0.2, random_state=RANDOM_SEED)

train_dataset = NewsDataset(X_train, s_train, y_train)
val_dataset = NewsDataset(X_val, s_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [49]:
# TextCNN

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes=1):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, out_channels=100, kernel_size=k) for k in [3, 4, 5]
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(100 * 3 + 1, num_classes)  # +1 для sentiment

    def forward(self, x_text, x_sentiment):
        x = self.embedding(x_text).transpose(1, 2)  # (B, E, L)
        x = [torch.relu(conv(x)).max(dim=2)[0] for conv in self.convs]  # max-pooling
        x = torch.cat(x, dim=1)
        x = torch.cat([x, x_sentiment], dim=1)  # добавляем sentiment как признак
        x = self.dropout(x)
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextCNN(vocab_size=len(vocab), embed_dim=100).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        texts = batch['text'].to(device)
        sentiments = batch['sentiment'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(texts, sentiments)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")

    # Валидация
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in val_loader:
            texts = batch['text'].to(device)
            sentiments = batch['sentiment'].to(device)
            labels = batch['label'].to(device)

            outputs = model(texts, sentiments)
            preds.extend((outputs > 0.5).int().cpu().numpy())
            truths.extend(labels.cpu().numpy())

    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds)
    print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

Epoch 1 | Loss: 0.7212
Accuracy: 0.5058 | F1 Score: 0.0214
Epoch 2 | Loss: 0.6983
Accuracy: 0.5099 | F1 Score: 0.6636
Epoch 3 | Loss: 0.6978
Accuracy: 0.5246 | F1 Score: 0.6247
Epoch 4 | Loss: 0.6914
Accuracy: 0.5214 | F1 Score: 0.6466
Epoch 5 | Loss: 0.6888
Accuracy: 0.5255 | F1 Score: 0.6553
Epoch 6 | Loss: 0.6785
Accuracy: 0.5304 | F1 Score: 0.6336
Epoch 7 | Loss: 0.6607
Accuracy: 0.5357 | F1 Score: 0.4592
Epoch 8 | Loss: 0.6367
Accuracy: 0.5454 | F1 Score: 0.5725
Epoch 9 | Loss: 0.5998
Accuracy: 0.5385 | F1 Score: 0.6223
Epoch 10 | Loss: 0.5594
Accuracy: 0.5425 | F1 Score: 0.5123
Epoch 11 | Loss: 0.5112
Accuracy: 0.5463 | F1 Score: 0.5724
Epoch 12 | Loss: 0.4642
Accuracy: 0.5422 | F1 Score: 0.5206
Epoch 13 | Loss: 0.4187
Accuracy: 0.5394 | F1 Score: 0.5023
Epoch 14 | Loss: 0.3848
Accuracy: 0.5382 | F1 Score: 0.4993
Epoch 15 | Loss: 0.3485
Accuracy: 0.5392 | F1 Score: 0.5864
Epoch 16 | Loss: 0.3204
Accuracy: 0.5426 | F1 Score: 0.5562
Epoch 17 | Loss: 0.2967
Accuracy: 0.5390 | F1 Sco

In [57]:
# BiGRU
class BiGRUClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, dropout=0.5):
        super(BiGRUClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=num_layers, 
                          batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2 + 1, 1)  # +1 для sentiment

    def forward(self, x_text, x_sentiment):
        embedded = self.embedding(x_text)  # (B, L, E)
        _, h_n = self.gru(embedded)        # h_n: (2, B, H)
        h_n = torch.cat((h_n[-2], h_n[-1]), dim=1)  # (B, 2H)
        x = torch.cat([h_n, x_sentiment], dim=1)    # добавим sentiment
        x = self.dropout(x)
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [61]:
model = BiGRUClassifier(vocab_size=len(vocab), embed_dim=100, hidden_dim=64).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        texts = batch['text'].to(device)
        sentiments = batch['sentiment'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(texts, sentiments)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")

    # Валидация
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in val_loader:
            texts = batch['text'].to(device)
            sentiments = batch['sentiment'].to(device)
            labels = batch['label'].to(device)

            outputs = model(texts, sentiments)
            preds.extend((outputs > 0.5).int().cpu().numpy())
            truths.extend(labels.cpu().numpy())

    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds)
    print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

Epoch 1 | Loss: 0.6951
Accuracy: 0.5164 | F1 Score: 0.5590
Epoch 2 | Loss: 0.6855
Accuracy: 0.5312 | F1 Score: 0.5407
Epoch 3 | Loss: 0.6690
Accuracy: 0.5446 | F1 Score: 0.5780
Epoch 4 | Loss: 0.6388
Accuracy: 0.5375 | F1 Score: 0.5089
Epoch 5 | Loss: 0.5938
Accuracy: 0.5394 | F1 Score: 0.5247
Epoch 6 | Loss: 0.5324
Accuracy: 0.5359 | F1 Score: 0.5633
Epoch 7 | Loss: 0.4648
Accuracy: 0.5357 | F1 Score: 0.5287
Epoch 8 | Loss: 0.3948
Accuracy: 0.5329 | F1 Score: 0.5454
Epoch 9 | Loss: 0.3266
Accuracy: 0.5350 | F1 Score: 0.5357
Epoch 10 | Loss: 0.2691
Accuracy: 0.5276 | F1 Score: 0.5202
Epoch 11 | Loss: 0.2243
Accuracy: 0.5297 | F1 Score: 0.5342
Epoch 12 | Loss: 0.1938
Accuracy: 0.5281 | F1 Score: 0.5187
Epoch 13 | Loss: 0.1687
Accuracy: 0.5275 | F1 Score: 0.5299
Epoch 14 | Loss: 0.1514
Accuracy: 0.5244 | F1 Score: 0.5351
Epoch 15 | Loss: 0.1339
Accuracy: 0.5274 | F1 Score: 0.5273
Epoch 16 | Loss: 0.1641
Accuracy: 0.5306 | F1 Score: 0.5221
Epoch 17 | Loss: 0.1361
Accuracy: 0.5229 | F1 Sco

In [65]:
# FastText-style
class FastTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(FastTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.fc = nn.Linear(embed_dim + 1, 1)  # +1 для sentiment

    def forward(self, x_text, x_sentiment):
        embedded = self.embedding(x_text)  # (B, L, E)
        mean_emb = embedded.mean(dim=1)    # усреднение по длине текста → (B, E)
        x = torch.cat([mean_emb, x_sentiment], dim=1)  # добавим sentiment
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [69]:
model = FastTextClassifier(vocab_size=len(vocab), embed_dim=100).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        texts = batch['text'].to(device)
        sentiments = batch['sentiment'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(texts, sentiments)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")

    # Валидация
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in val_loader:
            texts = batch['text'].to(device)
            sentiments = batch['sentiment'].to(device)
            labels = batch['label'].to(device)

            outputs = model(texts, sentiments)
            preds.extend((outputs > 0.5).int().cpu().numpy())
            truths.extend(labels.cpu().numpy())

    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds)
    print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

Epoch 1 | Loss: 0.6922
Accuracy: 0.5252 | F1 Score: 0.4430
Epoch 2 | Loss: 0.6870
Accuracy: 0.5360 | F1 Score: 0.5144
Epoch 3 | Loss: 0.6784
Accuracy: 0.5337 | F1 Score: 0.5476
Epoch 4 | Loss: 0.6654
Accuracy: 0.5379 | F1 Score: 0.4915
Epoch 5 | Loss: 0.6498
Accuracy: 0.5372 | F1 Score: 0.5815
Epoch 6 | Loss: 0.6334
Accuracy: 0.5422 | F1 Score: 0.5573
Epoch 7 | Loss: 0.6178
Accuracy: 0.5373 | F1 Score: 0.5485
Epoch 8 | Loss: 0.6025
Accuracy: 0.5349 | F1 Score: 0.5589
Epoch 9 | Loss: 0.5892
Accuracy: 0.5368 | F1 Score: 0.5136
Epoch 10 | Loss: 0.5764
Accuracy: 0.5386 | F1 Score: 0.5414
Epoch 11 | Loss: 0.5651
Accuracy: 0.5381 | F1 Score: 0.5463
Epoch 12 | Loss: 0.5550
Accuracy: 0.5372 | F1 Score: 0.5381
Epoch 13 | Loss: 0.5456
Accuracy: 0.5353 | F1 Score: 0.5480
Epoch 14 | Loss: 0.5368
Accuracy: 0.5326 | F1 Score: 0.5538
Epoch 15 | Loss: 0.5287
Accuracy: 0.5373 | F1 Score: 0.5364
Epoch 16 | Loss: 0.5212
Accuracy: 0.5316 | F1 Score: 0.5531
Epoch 17 | Loss: 0.5142
Accuracy: 0.5360 | F1 Sco

Усложним подход - добавим предобученные эмбеддинги из GloVe.

In [74]:
glove_path = 'glove.6B.100d.txt'

embedding_dim = 100
glove_embeddings = {}

with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        word = parts[0]
        vector = np.array(parts[1:], dtype=np.float32)
        glove_embeddings[word] = vector

# Создаём матрицу эмбеддингов под наш словарь
embedding_matrix = np.zeros((len(vocab), embedding_dim), dtype=np.float32)

for word, idx in vocab.items():
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Преобразуем в тензор и фиксируем <PAD> на ноль
embedding_matrix[0] = 0.0
embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float)

# Заменяем обычный nn.Embedding на предобученный
class FastTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(FastTextClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=False, padding_idx=0)
        self.fc = nn.Linear(embed_dim + 1, 1)

    def forward(self, x_text, x_sentiment):
        embedded = self.embedding(x_text)  # (B, L, E)
        mean_emb = embedded.mean(dim=1)    # (B, E)
        x = torch.cat([mean_emb, x_sentiment], dim=1)
        return torch.sigmoid(self.fc(x)).squeeze(1)

In [76]:
model = FastTextClassifier(vocab_size=len(vocab), embed_dim=100).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        texts = batch['text'].to(device)
        sentiments = batch['sentiment'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(texts, sentiments)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")

    # Валидация
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in val_loader:
            texts = batch['text'].to(device)
            sentiments = batch['sentiment'].to(device)
            labels = batch['label'].to(device)

            outputs = model(texts, sentiments)
            preds.extend((outputs > 0.5).int().cpu().numpy())
            truths.extend(labels.cpu().numpy())

    acc = accuracy_score(truths, preds)
    f1 = f1_score(truths, preds)
    print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

Epoch 1 | Loss: 0.6925
Accuracy: 0.5312 | F1 Score: 0.4922
Epoch 2 | Loss: 0.6869
Accuracy: 0.5307 | F1 Score: 0.6059
Epoch 3 | Loss: 0.6772
Accuracy: 0.5339 | F1 Score: 0.5702
Epoch 4 | Loss: 0.6630
Accuracy: 0.5402 | F1 Score: 0.5673
Epoch 5 | Loss: 0.6458
Accuracy: 0.5376 | F1 Score: 0.5896
Epoch 6 | Loss: 0.6287
Accuracy: 0.5412 | F1 Score: 0.5748
Epoch 7 | Loss: 0.6128
Accuracy: 0.5416 | F1 Score: 0.5523
Epoch 8 | Loss: 0.5980
Accuracy: 0.5384 | F1 Score: 0.5504
Epoch 9 | Loss: 0.5846
Accuracy: 0.5424 | F1 Score: 0.5441
Epoch 10 | Loss: 0.5721
Accuracy: 0.5373 | F1 Score: 0.5489
Epoch 11 | Loss: 0.5609
Accuracy: 0.5373 | F1 Score: 0.5395
Epoch 12 | Loss: 0.5511
Accuracy: 0.5389 | F1 Score: 0.5189
Epoch 13 | Loss: 0.5419
Accuracy: 0.5346 | F1 Score: 0.5370
Epoch 14 | Loss: 0.5331
Accuracy: 0.5368 | F1 Score: 0.5154
Epoch 15 | Loss: 0.5259
Accuracy: 0.5373 | F1 Score: 0.5417
Epoch 16 | Loss: 0.5180
Accuracy: 0.5375 | F1 Score: 0.5469
Epoch 17 | Loss: 0.5116
Accuracy: 0.5311 | F1 Sco

### Вывод по применению методов глубинного обучения

Цель данного этапа - оценить применимость простых моделей глубинного обучения для решения задачи классификации направления изменения цены акции на основе финансовых текстов. Мотивацией послужили ограниченные результаты, полученные с помощью классических алгоритмов машинного обучения, не способных эффективно учитывать контекст и последовательности слов, критически важные в анализе текстовой информации.

В рамках эксперимента были реализованы три лёгкие архитектуры нейронных сетей с использованием библиотеки PyTorch:  
- **TextCNN**,  
- **biGRU**,  
- **FastText-style**,  
а также модификация последней с использованием предобученных эмбеддингов **GloVe**.

Размер обучающего набора составил 50 000 строк, со сбалансированным таргетом. В качестве метрик использовались accuracy и F1 score.

#### Результаты после 20 эпох обучения

| Модель                         | Accuracy | F1 Score |
|-------------------------------|----------|----------|
| TextCNN                       | 0.5403   | 0.5533   |
| BiGRU                         | 0.5286   | 0.5365   |
| FastText-style                | 0.5347   | 0.5484   |
| FastText-style + GloVe        | 0.5335   | 0.5331   |


#### Анализ полученных результатов

Полученные значения метрик (accuracy около 53–54%, F1 score - до 55%) можно считать ожидаемыми для задачи высокой сложности. Финансовые тексты, как правило, содержат много шума, неоднозначностей, специфической терминологии и не всегда напрямую связаны с движением цены. Кроме того, даже для человека предсказание реакции рынка на новости зачастую затруднительно.

**Цель данного этапа достигнута**:  
- Продемонстрирована возможность применения моделей глубинного обучения для анализа финансовых текстов.  
- Выстроен процесс предобработки текстовых данных.  
- Подтверждено, что даже простые архитектуры глубинного обучения более чувствительны к структуре и смыслу текстов, чем классические ML-модели, что находит отражение в улучшенных значениях метрик качества.

#### Возможности для дальнейших улучшений

При наличии вычислительных и временных ресурсов возможны следующие направления развития:  
- Предобучение языковых моделей (например, BERT, FinBERT, RoBERTa) на собственных финансовых данных.  
- Введение дополнительных рыночных признаков (волатильность, объёмы торгов, технические индикаторы).  
- Применение мультимодальных моделей, объединяющих текстовые данные, графики и временные ряды.

Методы глубинного обучения показали большую эффективность в задаче предсказания направления изменения цены акций на основе финансовых новостей. Полученные результаты свидетельствуют о  наличии потенциала дальнейшего улучшения качества классификации при применении более сложных архитектур и расширении объёма обучающих данных.